In [176]:
import pandas as pd

ratings_df = pd.read_csv('data/ratings_movies.csv')

ratings_df.head(10)

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
    
#У скольких фильмов не указан год их выпуска?    
ratings_df['year_release'] = ratings_df['title'].apply(get_year_release)

#Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
low1999 = ratings_df[ratings_df['year_release'] == 1999]

low1999 = low1999.sort_values(by = ['rating'], ignore_index= True)

low1999 = low1999.groupby(['title'])['rating'].agg('mean').sort_values()

low1999.head()

#Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
low2010= ratings_df[ratings_df['year_release'] == 2010]
low2010 = low2010.groupby(['genres'])['rating'].agg('mean').sort_values()

low2010.head()

#Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
userId_df = ratings_df.groupby(['userId']).agg({'genres': 'nunique'}).sort_values(by = ['genres'], ascending= False)
#или
userId_df = ratings_df.groupby(['userId'])['genres'].agg('nunique').sort_values(ascending= False)
userId_df.head(1000)

#Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
LowuserId_df = ratings_df.groupby(['userId'])['rating'].agg(['count','mean'])
LowuserId_df = LowuserId_df[LowuserId_df['count'] == 20].sort_values(by = ['mean'], ascending= False)
LowuserId_df.head()
print(LowuserId_df.index[0])

#Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
#и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

max_genres = ratings_df[ratings_df['year_release'] == 2018]
max_genres = max_genres.groupby(['genres'])['rating'].agg(['count'])
max_genres = max_genres[max_genres['count'] > 10]
max_genres.sort_values(by = ['count'])

max_genres.head(10)

#Добавьте в таблицу новый признак year_rating — год выставления оценки. 
#Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.
ratings_df['year_rating'] = pd.to_datetime(ratings_df['date'])
ratings_df['year_rating'] = ratings_df['year_rating'].dt.year
pivot_df = ratings_df.pivot_table(values='rating',index='year_rating',columns='genres',aggfunc='mean')
pivot_df = pivot_df.fillna(0)

#За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
print('1 - ',pivot_df['Action|Adventure'].min())
#Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
print('2 - ',pivot_df['Action|Adventure|Animation|Children|Comedy|IMAX'].idxmax())
#Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.
#pivot_df.head()
#print('3 -',pivot_df.loc[[2018]].sort_values(by = 2018, axis = 1, ascending = False))

#list1 = pivot_df[pivot_df['year_release'] == 2018]

#Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).
print('4 - ',pivot_df['Comedy'])


53
1 -  3.2777777777777777
2 -  2013
4 -  year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64
